<a href="https://colab.research.google.com/github/perlatomdpi/NLP/blob/main/BERT_Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BERT Tokenizer**
Example of tokenization using BERT



# **Import dependencies**


In [1]:
import numpy as np
import math
import re # advanced text pre-processing
import pandas as pd
from bs4 import BeautifulSoup # encode text
import random

from google.colab import drive # get data from drive

In [2]:
# packages related to BERT
!pip install bert-for-tf2 
!pip install sentencepiece # requirement for tf2

     |████████████████████████████████| 51kB 5.7MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.7-cp36-none-any.whl size=30537 sha256=6b45179b2cf36a93d547d8868a5b9181d83860fb70afb655bc6b2becd10c379b
  Stored in directory: /root/.cache/pip/wheels/e1/f8/e2/b98f79a6b8cc898d8e4102b83acb8a098df7d27500a2bac912
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7303 sha256=75708213348abb1a844e534e1ea42a284b50abe0a62a98ecb492a86c7ee9fe87
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19474 sha256=61c0e1fb062f9013f333b65c5c1f1bdaa1e3323dad67a2728f189ff93699783d
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.1MB 6.8MB/s 


In [7]:
# use tf2
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub # to download the weights of bert
from tensorflow.keras import layers # to create cnn layers
import bert

# **Data pre-processing**


In [8]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
cols = ["sentiment", "id", "query", "user", "text"]
data = pd.read.csv(
    "content/drive/My Drive/.../BERT/data/train.csv",
    header=None,
    names=cols,
    engine="python", 
    encoding="latin1"
)

In [ ]:
data.drop(["id", "date", "query", "user"],
          axis=1,
          inplace=True) # inplace garantee that new lighter data are loaded

# **Cleaning**

In [ ]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    tweet = re.sub(r"@[A-Za-z0-9]+", '',tweet)   # replace a substring with another
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", '', tweet)
    tweet = re.sub(r"[^A-Za-z.!?]", '', tweet)
    tweet = re.sub(r" +", '', tweet)
    return tweet

In [ ]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

In [ ]:
data_labels = data.sentiment.values # get the value of the sentiment
data_labels[data_labels == 4] = 1   # in the data 4 is for positve and we converted it to 1

# **Tokenization**
We have to create a BERT layer to have access to meta data for the tokenier  - vocab size.

In [ ]:
FullTokenizer = bert.bert_tokenization.FullTokenizer # call bert module

bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False) # create bert layer - hub is were the pre-trainined model is stored
                                             # here we use the light version of bert
                                             # L-12: 12 encoders
                                             # trainable=false --> we don't use bert for fine-tuning weights

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy() # give access to the vocab file as numpy
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy() # lower case
tokenizer = FullTokenizer(vocab_file, do_lower_case) # create the tokenizer